In [1]:
import numpy as np
import pandas as pd
import requests 
from bs4 import BeautifulSoup
from fake_useragent import UserAgent

from urllib.request import urlopen
import urllib

from selenium import webdriver
import openpyxl
import time
import math
from collections import Counter


from tqdm.notebook import tqdm
import json


In [20]:
def url_split(url):
    main_,para_1 = url.split("?")
    para_2 = para_1.split("&")
    para_3 = [d.replace("=",":").split(":") for d in para_2 if len(d.replace("=",":").split(":")) != 1]
    params_ = {}
    #print(para_3)
    for key,value in para_3:
        params_[key] = value
    return main_,params_

# Main Page

In [21]:
ua = UserAgent()

In [22]:
def Main_page_head(q_1,p_1):
    url_2 ="https://search.tmon.co.kr/api/search/v4/deals?_=1622950347304&keyword={}&mainDealOnly=true&optionDealOnly=false&page={}&showFilter=false&size=60&sortType=REVIEW&thr=hs&useTypoCorrection=true"

    html = requests.get(url_2.format(q_1,p_1)).text
    review_1 = json.loads(html)

    item_list,name_head,reveiw_count_1 = [],[],[]
    for Ta_ in range(len(review_1["data"]["searchDeals"])):
        item_list.append(review_1["data"]["searchDeals"][Ta_]["searchDealResponse"]["searchInfo"]["id"])
        name_head.append(review_1["data"]["searchDeals"][Ta_]["searchDealResponse"]["dealInfo"]["titleName"])
        reveiw_count_1.append(review_1["data"]["searchDeals"][Ta_]["searchDealResponse"]["dealInfo"]["userPoint"]["reviewCount"])
    return pd.DataFrame([name_head,item_list,reveiw_count_1]).T.rename(columns={0:"Name",1:"item_num",2:"Review_count"})

In [23]:
main_df = pd.concat([Main_page_head("헤어드라이기",1),Main_page_head("헤어드라이기",2)]).reset_index(drop=True)
main_df

,Name,item_num,Review_count
0,[아이디어라떼] 끝판왕시리즈 우리집 아이디어 생활용품 339종 모음전,2719804166,2714
1,[특가] 인블룸 가을맞이 리빙/욕실/청소 필요한 모든것 94종 창고대방출,2515964826,2229
2,[CKI] 인기 드라이기 모음,463736766,1586
3,[제니아] 2200W 드라이기 템페스트 WJD-5560,2232240862,1166
4,[유닉스] 드라이기 UN-A1610,605332150,901
...,...,...,...
115,욕실 수납장 홀더 선반,1940666614,20
116,[CKI] D210 드라이기,642088722,20
117,다이슨 슈퍼소닉 헤어 드라이기 스크래치 방지 실리콘 커버 케이스,3686099754,19
118,주방 싱크대 칼꽂이 선반 조리도구 국자 조리기 걸이,2512115938,19


In [24]:
p_2 = 1

Ta_ = 0

item_n = main_df["item_num"][Ta_]
target_name = main_df["Name"][Ta_]
max_page = int(main_df["Review_count"][Ta_])//200

# Review Page

In [25]:
def Review_page_ex(target_name,item_n,p_2):
    url_3 = "http://www.tmon.co.kr/api/detail/direct/v1/boardapi/api/review/search/list/{}?offset=200&page={}&reviewNo=&useSc=false&timeout=5000&useHeader=&image=Y&count=20"
    header = {'User-Agent':ua.chrome,
              "referer":"https://www.tmon.co.kr/"}

    html = requests.get(url_3.format(item_n,p_2),headers=header).text
    review_2 = json.loads(html)
    data_all = []
    for Ta_3 in range(len(review_2["data"]["list"])):
        id_ = review_2["data"]["list"][Ta_3]["whoCreate"]
        date_ = review_2["data"]["list"][Ta_3]["createDt"]
        item_ = target_name
        score_ = int(review_2["data"]["list"][Ta_3]["dealGpoint"])
        contents_ = review_2["data"]["list"][Ta_3]["contents"]
        liked_ = review_2["data"]["list"][Ta_3]["recommendCnt"]
        channel_ = "TIMON"
        all_ = [date_,id_,item_,score_,contents_,liked_,channel_]
                
        data_all.append(all_)
    df_ = pd.DataFrame(data_all).rename(columns={0:"date",1:"id",2:"item",3:"score",4:"contents",5:"liked",6:"channel"})
    return df_

In [26]:
Ta_ = 1
item_n = main_df["item_num"][Ta_]
target_name = main_df["Name"][Ta_]
max_page = main_df["Review_count"][Ta_]

Review_page_ex(target_name,item_n,1).head()

,date,id,item,score,contents,liked,channel
0,2020-09-02 23:03:02,19j******,[특가] 인블룸 가을맞이 리빙/욕실/청소 필요한 모든것 94종 창고대방출,5,옷걸이 좋아요 가성비 좋아서 샀아요,0,TIMON
1,2020-09-02 18:45:41,fuo*****,[특가] 인블룸 가을맞이 리빙/욕실/청소 필요한 모든것 94종 창고대방출,4,일본제품을 써봣는데 의외로 소소하네욤 딱딱한 재질인걸보고 놀랬습니다,0,TIMON
2,2020-08-28 18:21:37,chi****,[특가] 인블룸 가을맞이 리빙/욕실/청소 필요한 모든것 94종 창고대방출,4,다리가 짱짱해서 만족합니다.,0,TIMON
3,2020-08-13 22:37:43,kim******,[특가] 인블룸 가을맞이 리빙/욕실/청소 필요한 모든것 94종 창고대방출,5,간단하게붙여서쓰기에좋아요\n다만딱고정이안되고 살짝 불안하네용,0,TIMON
4,2020-08-07 17:15:01,beb*****,[특가] 인블룸 가을맞이 리빙/욕실/청소 필요한 모든것 94종 창고대방출,3,좋아요 침대밑에 딱 맞네요\n진작 구입할껄 했어요\n색상도 이쁘고 디자인도 좋아요,0,TIMON


In [27]:
def Review_page_ex2(target_name,item_n,p_2):
    url_3 = "http://www.tmon.co.kr/api/detail/direct/v1/detailapi/api/detail/board/review/list/{}?categoryNo=57000000&page={}&count=20&useHeader=&flagSc=N"
    header = {'User-Agent':ua.chrome,
              "referer":"https://www.tmon.co.kr/"}

    html = requests.get(url_3.format(item_n,p_2),headers=header).text
    review_2 = json.loads(html)
    data_all = []

    for Ta_3 in range(len(review_2["data"]["reviewContents"])):
        id_ = review_2["data"]["reviewContents"][Ta_3]["whoUpdate"]
        date_ = review_2["data"]["reviewContents"][Ta_3]["createDt"]
        item_ = target_name
        score_ = int(review_2["data"]["reviewContents"][Ta_3]["dealGpoint"])
        contents_ = review_2["data"]["reviewContents"][Ta_3]["contents"]
        liked_ = review_2["data"]["reviewContents"][Ta_3]["recommendCnt"]
        channel_ = "TIMON"
        all_ = [date_,id_,item_,score_,contents_,liked_,channel_]
        data_all.append(all_)
    df_ = pd.DataFrame(data_all).rename(columns={0:"date",1:"id",2:"item",3:"score",4:"contents",5:"liked",6:"channel"})
    return df_

In [28]:
Ta_ = 1
item_n = main_df["item_num"][Ta_]
target_name = main_df["Name"][Ta_]
max_page = main_df["Review_count"][Ta_]

Review_page_ex2(target_name,item_n,1).head()

,date,id,item,score,contents,liked,channel
0,2021-07-12 19:30:00,ghk*********,[특가] 인블룸 가을맞이 리빙/욕실/청소 필요한 모든것 94종 창고대방출,5,양치 하고도 이에 낀 음식물 안빠질 때!<br />개운하고 잘 빠집니다~예전에 막 ...,0,TIMON
1,2021-06-29 10:49:53,s2e****,[특가] 인블룸 가을맞이 리빙/욕실/청소 필요한 모든것 94종 창고대방출,5,좋아용 잘쓸게용ㅇㅇㅇ,0,TIMON
2,2021-06-16 14:58:36,Gs2***,[특가] 인블룸 가을맞이 리빙/욕실/청소 필요한 모든것 94종 창고대방출,5,조아요조아요조아용조아용,0,TIMON
3,2021-05-30 20:05:14,sh4***,[특가] 인블룸 가을맞이 리빙/욕실/청소 필요한 모든것 94종 창고대방출,3,좋아요좋아요좋아요좋아요좋아요,0,TIMON
4,2021-05-21 09:23:46,pyj****,[특가] 인블룸 가을맞이 리빙/욕실/청소 필요한 모든것 94종 창고대방출,5,좋아서 더 주문해써용 많이파세요,0,TIMON


In [29]:
def Crawling_Timon_all(main_df,Ta_):
    item_n = main_df["item_num"][Ta_]
    target_name = main_df["Name"][Ta_]
    max_page = main_df["Review_count"][Ta_]

    MaxPage = max_page//200
    tq = tqdm(range(MaxPage+1))

    review_all = pd.DataFrame()
    for mp in tq:
        review_candi = pd.concat([Review_page_ex(target_name,item_n,mp+1),Review_page_ex2(target_name,item_n,mp+1)]).reset_index(drop=True)
        review_all = pd.concat([review_all,review_candi]).reset_index(drop=True)
    return review_all,target_name

In [30]:
test_1,name_1 = Crawling_Timon_all(main_df,1)

  0%|          | 0/12 [00:00<?, ?it/s]

In [31]:
def Timon_Save_local(path_,head_df):
    for i_ in range(len(head_df)):
        print("In {} Pro From {}".format(i_+1,len(head_df)+1))
        print()
        print(head_df["Name"][i_])
        out_,NA_ = Crawling_Timon_all(head_df,i_)
        NA_ = NA_.replace("/","").replace("-","").replace("\n","").strip()
        out_.to_excel(path_.format(NA_))

In [32]:
path_ = r"C:\Users\Samsung\jupyter\Project_LG_textmining\data\hair\shopping_2\TIMON\{}.xlsx"
Timon_Save_local(path_,main_df)

In 1 Pro From 121

[아이디어라떼] 끝판왕시리즈 우리집 아이디어 생활용품 339종 모음전


  0%|          | 0/14 [00:00<?, ?it/s]

In 2 Pro From 121

[특가] 인블룸 가을맞이 리빙/욕실/청소 필요한 모든것 94종 창고대방출


  0%|          | 0/12 [00:00<?, ?it/s]

In 3 Pro From 121

[CKI] 인기 드라이기 모음


...

  0%|          | 0/1 [00:00<?, ?it/s]

In 119 Pro From 121

주방 싱크대 칼꽂이 선반 조리도구 국자 조리기 걸이


  0%|          | 0/1 [00:00<?, ?it/s]

In 120 Pro From 121

주방 싱크대 국자 조리도구 행주 벽 걸이 건조대 선반


  0%|          | 0/1 [00:00<?, ?it/s]